https://archive.ics.uci.edu/ml/datasets/HTRU2

In [303]:
import pandas as pd
import numpy as np
data = pd.read_csv("HTRU_2.csv", header=None)
data[8].value_counts()

0    16259
1     1639
Name: 8, dtype: int64

Взял не сбалансированный сет, на уроке показалось, что с таким должны быть проблемы, так как наша схема хорошо должна себя проявить на половине данных. В процессе станет понятно, если что уберу часть данных, сделаю сет сбалансированным.

In [304]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       17898 non-null  float64
 1   1       17898 non-null  float64
 2   2       17898 non-null  float64
 3   3       17898 non-null  float64
 4   4       17898 non-null  float64
 5   5       17898 non-null  float64
 6   6       17898 non-null  float64
 7   7       17898 non-null  float64
 8   8       17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


In [305]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
0,17898.0,111.079968,25.652935,5.812500,100.929688,115.078125,127.085938,192.617188
1,17898.0,46.549532,6.843189,24.772042,42.376018,46.947479,51.023202,98.778911
2,17898.0,0.477857,1.064040,-1.876011,0.027098,0.223240,0.473325,8.069522
3,17898.0,1.770279,6.167913,-1.791886,-0.188572,0.198710,0.927783,68.101622
4,17898.0,12.614400,29.472897,0.213211,1.923077,2.801839,5.464256,223.392140
5,17898.0,26.326515,19.470572,7.370432,14.437332,18.461316,28.428104,110.642211
6,17898.0,8.303556,4.506092,-3.139270,5.781506,8.433515,10.702959,34.539844
7,17898.0,104.857709,106.514540,-1.976976,34.960504,83.064556,139.309331,1191.000837
8,17898.0,0.091574,0.288432,0.000000,0.000000,0.000000,0.000000,1.000000


Данные полные, категориальных данных нету. Все данные вещественные. Так кк модель будет не линейной, то нормализация,стандартизация нам не нужна.

In [306]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [307]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier()

cat.fit(x_train, y_train, verbose=False)
y_predict = cat.predict(x_test)
y_predict.shape

(3580,)

In [308]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 88.49%
roc: 92.36%
recall: 85.55%
precision: 91.64%


Получилось вроде норм

In [309]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.1 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 164/1639 as positives and unlabeling the rest


In [310]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    17734
 1      164
Name: class_test, dtype: int64


In [311]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [312]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(164, 10) (164, 10)


In [313]:
sample_test.shape

(17570, 10)

In [314]:
cat = CatBoostClassifier()
cat.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values, verbose=False)
y_predict = cat.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 77.02%
roc: 93.55%
recall: 91.26%
precision: 66.62%


In [315]:
np.bincount(y_predict)

array([15563,  2007], dtype=int64)

Соберем из этого подобие кросс валидации, так как выглядит как будто в процессе есть большая дисперсия.

In [316]:
def metrics_list(y_test, y_predict):
    f1 = f1_score(y_test, y_predict)
    roc = roc_auc_score(y_test, y_predict)
    rec = recall_score(y_test, y_predict, average='binary')
    prc = precision_score(y_test, y_predict, average='binary')
    return [f1, roc, rec, prc]

def some_fake_cv(data, cv=4, percent=0.1, verbose=False):
    '''
    data - our dataset
    cv  - number of models for validation
    percent - percent of positive samples have been taken

    Return: list with metrics lists: [f1, roc, recall, precision]
    '''

    results = []
    for i in range(cv):
        mod_data = data.copy()
        pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
        np.random.shuffle(pos_ind)
        pos_sample_len = int(np.ceil(percent * len(pos_ind)))
        pos_sample = pos_ind[:pos_sample_len]
        mod_data['class_test'] = -1
        mod_data.loc[pos_sample,'class_test'] = 1
        x_data = mod_data.iloc[:,:-2].values # just the X 
        y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
        y_positive = mod_data.iloc[:,-2].values # original class
        mod_data = mod_data.sample(frac=1)
        neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
        sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
        pos_sample = mod_data[mod_data['class_test']==1]
        sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
        cat = CatBoostClassifier()
        cat.fit(sample_train.iloc[:,:-2].values, 
                sample_train.iloc[:,-2].values, verbose=False)
        y_predict = cat.predict(sample_test.iloc[:,:-2].values)
        results.append(metrics_list(sample_test.iloc[:,-2].values, y_predict))
    if verbose:
        print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
        print('target variable:\n', mod_data.iloc[:,-1].value_counts())
        print(neg_sample.shape, pos_sample.shape)
    return np.array(results)


In [317]:
cv_results = some_fake_cv(data, cv=8, percent=0.1, verbose=True)

Using 164/1639 as positives and unlabeling the rest
target variable:
 -1    17734
 1      164
Name: class_test, dtype: int64
(164, 10) (164, 10)


In [318]:
def print_cv_results(cv_results, cv=1):
    print('Classification results:')
    for i, score in enumerate(('f1', 'roc', 'recall', 'precision')):
        print(f'{score}: {cv_results[i].mean()*100:.3f}% +- {cv_results[i].std()*100/np.sqrt(cv):.3f}%')
    print()

In [319]:
print_cv_results(cv_results, cv=8)

Classification results:
f1: 83.143% +- 3.332%
roc: 82.716% +- 3.094%
recall: 83.361% +- 3.190%
precision: 83.310% +- 3.362%



тут погрешность за 8 опытов, посомтрим на стандартное отклонение.

In [322]:
print_cv_results(cv_results, cv=1)

Classification results:
f1: 83.143% +- 9.424%
roc: 82.716% +- 8.750%
recall: 83.361% +- 9.023%
precision: 83.310% +- 9.509%



По полученным результатам мы видим, что дисперсия чуть больше чем чудовищная. Сделаем побольше опытов что бы посмотреть среднее значение метрика работы такого классификатора.

In [320]:
%%time
from tqdm import notebook
percents = np.linspace(0.05, 0.7, 14)
cv = 50
results_for_graph = dict()
for percent in notebook.tqdm(percents):
    results_for_graph[percent] = some_fake_cv(data, cv=cv, percent=percent, verbose=False)


Wall time: 38min 39s


In [321]:
for percent in percents:
    print(f'Percent of positive class samples have been taken: {percent}')
    print_cv_results(results_for_graph[percent], cv=cv)

Percent of positive class samples have been taken: 0.05
Classification results:
f1: 77.373% +- 2.205%
roc: 75.738% +- 2.470%
recall: 81.956% +- 1.501%
precision: 84.184% +- 1.049%

Percent of positive class samples have been taken: 0.1
Classification results:
f1: 83.218% +- 1.451%
roc: 73.702% +- 2.639%
recall: 80.996% +- 1.742%
precision: 83.169% +- 1.368%

Percent of positive class samples have been taken: 0.15
Classification results:
f1: 81.597% +- 1.529%
roc: 84.106% +- 1.239%
recall: 81.873% +- 1.498%
precision: 81.561% +- 1.739%

Percent of positive class samples have been taken: 0.2
Classification results:
f1: 80.964% +- 1.784%
roc: 79.671% +- 2.053%
recall: 81.234% +- 1.755%
precision: 81.349% +- 1.817%

Percent of positive class samples have been taken: 0.25
Classification results:
f1: 80.057% +- 1.946%
roc: 81.235% +- 1.795%
recall: 81.711% +- 1.686%
precision: 81.517% +- 1.666%

Percent of positive class samples have been taken: 0.3
Classification results:
f1: 79.596% +- 2.0

Единственно что видно, что при увеличении размера первоначальной положительной выборки качество начинает падать. Скоерй всего из за дисбаланса классов, когда мы берем почти все данные позитивного класса, мы обучаем соседнюю с ним негаивную зону, чем сильно уменьшаем наши характеристики добавляя много неправильнвых ответов в наша класс.

Вы рассказывал и что более модным является 2-step approach. видимо он более переспективен, но скорей всего им можно перобучиться так как он более сложный.